In [1]:
# Importação de bibliotecas
import pandas as pd

from pendulum import now

In [2]:
# Objetos
fonte = 'sidra'
dataset_name = 'tab3939_dejetos'
variavel = 'cabecas'

raiz = '/home/eduardo/repositorios/IS_Agro/databases'

### Bronze

In [ ]:
contador = 1

ultimo_ano = now().year - 1
for ano in range(2000, ultimo_ano):
    if contador == 1:
        df_3939 = pd.read_json(f'https://apisidra.ibge.gov.br/values/t/3939/n6/all/v/all/p/{ano}/c79/32793,32794,32796')
        contador += 1
    else:
        df_3939 = pd.concat(
            [df_3939,
             (pd.read_json(f'https://apisidra.ibge.gov.br/values/t/3939/n6/all/v/all/p/{ano}/c79/32793,32794,32796')
              .iloc[1:])],
            ignore_index=True
            )
        contador += 1

df_3939.to_json(f'{raiz}/bronze_{fonte}_{dataset_name}_{variavel}.json')

### Prata

|  CÓDIGO |        CULTURA        |
|---------|-----------------------|
| 32793   | Galináceos - galinhas |
| 327930  | Galináceos - frangos  |
| 32794   |  Suínos - total       |
| 32796   | Galináceos - total    |

In [7]:
df = (pd.read_json(f'{raiz}/bronze_{fonte}_{dataset_name}_{variavel}.json')
 .drop(0, axis = 0)
 .filter(['D1C', 'D3C', 'D4C', 'V'])
 .query('D4C in ("32793", "32794", "32796")')
 .reset_index(drop=True)
 .assign(**{
     'V': lambda df: pd.to_numeric(df['V'], errors='coerce')
     })
    )

df_frangos = (pd.DataFrame()
             .assign(**{
                 'D1C': df.iloc[1::3, 0].reset_index(drop=True),
                 'D3C': df.iloc[1::3, 1].reset_index(drop=True),
                 'D4C': '327930',
                 'V': df.iloc[2::3, 3].reset_index(drop=True) - df.iloc[::3, 3].reset_index(drop=True)
             })
            )

(pd.concat(
    [df.query('D4C != "32796"'),
     df_frangos]
    )
 .assign(**{
        'uf': lambda df: df['D1C'].str.slice(0,2).astype(int),
        'kg_cabeca': lambda df: pd.to_numeric(df['V'], errors='coerce'),
        'municipio': lambda df: df['D1C'].astype(int),
        'data': lambda df: pd.to_datetime(df['D3C'], format='%Y'), # Coluna "data" em datetime baseado na ISO 8601, convertido da coluna "D3C"
        'cultura': lambda df: df['D4C'].astype(int),
        'fonte': 'Dejetos'
        })
 .dropna(subset='kg_cabeca')
 .convert_dtypes()
 .groupby(['uf', 'municipio', 'data', 'cultura', 'fonte'])
 .agg({'kg_cabeca':'sum'})
 .to_parquet(f'{raiz}/prata_{fonte}_{dataset_name}_{variavel}.parquet')
 )

### Ouro

Cálculos:

N: ((kg_cabeça * Teor N)/1000) * ((100 - perdas_N)/100)

P, K: (kg_cabeca * Teor N, K)/1000

In [3]:
parametro = {
    32793: {'n': 0.58, 'p': 0.18, 'k': 0.21}, # de Galinhas
    327930: {'n': 0.4, 'p': 0.12, 'k': 0.24}, # de Frangos
    32794: {'n': 14.2, 'p': 2.3, 'k': 6.2}, # Suínos
}

# Perdas por volatilização do N
perdas_N_galinaceos = 0
perdas_N_suinos = 0

In [8]:
df = (pd.read_parquet(f'{raiz}/prata_{fonte}_{dataset_name}_{variavel}.parquet')
 .groupby(['uf', 'data', 'cultura', 'fonte'])
 .agg({'kg_cabeca': 'sum'})
 .reset_index()
 .assign(
      N = lambda df: df.apply(lambda lin: ((lin['kg_cabeca'] * parametro[lin['cultura']]['n'])/1000) * ((100 - perdas_N_suinos)/100) 
          if lin['cultura'] == 32794 else
          ((lin['kg_cabeca'] * parametro[lin['cultura']]['n'])/1000) * ((100 - perdas_N_galinaceos)/100), axis = 1),
      P= lambda df: df.apply(lambda lin: (lin['kg_cabeca'] * parametro[lin['cultura']]['p'])/1000, axis = 1),
      K= lambda df: df.apply(lambda lin: (lin['kg_cabeca'] * parametro[lin['cultura']]['k'])/1000, axis = 1),
      fonte = lambda df: df['cultura'].apply(lambda x: 'Dejetos suínos' if x == 32794 else 'Dejetos galináceos')
  )
 .drop('kg_cabeca', axis = 1)
 .melt(id_vars=['uf', 'data', 'cultura', 'fonte'],
       value_vars=['N', 'P', 'K'],
       var_name='elemento',
       value_name='valor')
 .convert_dtypes()
 .groupby(['uf', 'data', 'fonte', 'elemento'])
 .agg({'valor': 'sum'})
 .reset_index(level='elemento')
 .sort_values(by=['elemento', 'uf'],
                   key=lambda x: x.map({'N': 0, 'P': 1, 'K': 2}))
 )

In [12]:
df = (
    df
    .rename_axis(index={
        'uf':'state',
        'data': 'date',
        'fonte': 'label'
    })
    .rename(columns={
        'elemento':'nutrient',
        'valor':'value'
    })
)

df.to_csv(f'{raiz}/ouro_npk_dejetos.csv')